In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()
args["is_slippery"] = True
args["ep_max_steps"] = 100
args["slippery_mode"] = "extreme"

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"], slipper_mode=args["slippery_mode"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [3]:
args["update_method"] = "avg"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")

Q_mcts_dict = defaultdict()


hparam_ucb_scale_list = np.arange(10, 100, 10)
# hparam_ucb_scale_list = np.arange(20, 64, 4)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]


num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
# num_trajectories_list = [200, 500, 800]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
start_time = time.time()
for num_trajectories in num_trajectories_list[3:]:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list:
        # start_time = time.time()

        # print(f"hparam_ucb_scale = {hparam_ucb_scale}")
        args["hparam_ucb_scale"] = hparam_ucb_scale
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_std:0.2f}) "
        # print(f"reward = {reward_mean:0.2f} +/- {reward_std:0.2f}")

        Q_mcts_dict[f"{hparam_ucb_scale}"] = copy.deepcopy(Q_mcts_list)

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_ucb_scale
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")
    
    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 20

-> num_trajectories = 1500

-> num_trajectories = 2000

-> num_trajectories = 2500

-> num_trajectories = 3000


In [4]:
print(res_text1)
print(res_text2)

1500 & -104.40 (±3.82) & -104.90 (±4.06) & -99.85 (±20.94) & -99.50 (±19.96) \\ 
 \hline 
2000 & -95.55 (±31.05) & -104.30 (±2.90) & -105.50 (±4.05) & -93.95 (±29.86) \\ 
 \hline 
2500 & -104.25 (±2.71) & -103.60 (±2.60) & -98.85 (±21.76) & -94.30 (±29.79) \\ 
 \hline 
3000 & -99.55 (±21.36) & -99.55 (±22.48) & -96.65 (±31.02) & -93.70 (±29.71) \\ 
 \hline 

1500 & -100.10 (±21.82) & -98.55 (±21.47) & -99.55 (±21.44) & -85.90 (±40.11) \\ 
 \hline 
2000 & -98.90 (±21.92) & -104.15 (±3.51) & -98.55 (±21.98) & -96.15 (±29.62) \\ 
 \hline 
2500 & -93.60 (±30.43) & -99.50 (±22.52) & -103.60 (±3.25) & -100.45 (±22.20) \\ 
 \hline 
3000 & -89.60 (±34.43) & -100.35 (±22.25) & -103.95 (±3.36) & -104.20 (±2.24) \\ 
 \hline 

